# Hadoop Streaming assignment 1: Words Rating

The purpose of this task is to create your own WordCount program for Wikipedia dump processing and learn basic concepts of the MapReduce.

In this task you have to find the 7th word by popularity and its quantity in the reverse order (most popular first) in Wikipedia data (`/data/wiki/en_articles_part`).

There are several points for this task:

1) As an output, you have to get the 7th word and its quantity separated by a tab character.

2) You must use the second job to obtain a totally ordered result.

3) Do not forget to redirect all trash and output to /dev/null.

Here you can find the draft of the task main steps. You can use other methods for solution obtaining.

## Step 1. Create mapper and reducer.

<b>Hint:</b>  Demo task contains almost all the necessary pieces to complete this assignment. You may use the demo to implement the first MapReduce Job.

In [1]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    index = 0
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print >> sys.stderr, "reporter:status:processed %d words" % index
        index += 1
        #print (word.lower(),1,sep="\t")
        print "%s\t%d" % (word.lower(),1)

Overwriting mapper1.py


In [2]:
%%writefile reducer1.py

#from __future__ import print_function
import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count =line.strip().split('\t',1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
            #print(current_key, word_sum,sep="\t")
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)
    #print(current_key, word_sum,sep="\t") '''   

Overwriting reducer1.py


In [3]:
# You can use this cell for other experiments: for example, for combiner.

## Step 2. Create sort job.

<b>Hint:</b> You may use MapReduce comparator to solve this step. Make sure that the keys are sorted in ascending order.

In [4]:
%%writefile mapper2.py

import sys

for line in sys.stdin:
    try:
        key, count =line.strip().split('\t',1)
        count = int(count)
    except ValueError as e:
        continue
    print "%d\t%s\t%d" % (count,key,0)
    
# Your code for sort job here. Don't forget to use magic writefile

Overwriting mapper2.py


In [5]:
%%writefile reducer2.py
import sys

for line in sys.stdin:
    try:
        count, key,temp =line.strip().split('\t',2)
        count = int(count)
    except ValueError as e:
        continue
    print "%s\t%d" % (key,count)

Overwriting reducer2.py


## Step 3. Bash commands

<b> Hint: </b> For printing the exact row you may use basic UNIX commands. For instance, sed/head/tail/... (if you know other commands, you can use them).

To run both jobs, you must use two consecutive yarn-commands. Remember that the input for the second job is the ouput for the first job.

In [6]:
%%bash

OUT_DIR="assignment1_"$(date +"%s%6N")
STREAMING_JAR="/opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar"
STREAMING_JAR="/usr/local/hadoop-2.8.1/hadoop-streaming.jar"

hdfs dfs -rm -r -skipTrash ${OUT_DIR}* > /dev/null
# Code for your first job
# yarn jar /usr/local/hadoop-2.8.1/hadoop-streaming.jar \
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Streaming wordrating" \
    -D mapreduce.job.maps=2 \
    -D mapreduce.job.reduces=8 \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -reducer "python reducer1.py" \
    -combiner "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null
    
    #-D mapreduce.partition.keycomparator.option=-k1,1 -k2,2nr

# Code for your second job

OUT_DIR2="Fassignment1_"$(date +"%s%6N")

# yarn jar /usr/local/hadoop-2.8.1/hadoop-streaming.jar \
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Streaming wordrating 2" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.map.output.key.field.separator=\t \
    -D mapreduce.partition.keycomparator.options="-k1,1nr" \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python reducer2.py" \
    -input ${OUT_DIR}/part-* \
    -output ${OUT_DIR2} > /dev/null


#     -D mapreduce.job.maps=2 \
#     -D stream.map.output.field.separator=. \
#     -D stream.num.map.output.key.fields=3 \
#     -D stream.reduce.input.field.separator=. \
#     -D stream.reduce.output.field.separator=. \
#     -D stream.num.reduce.output.key.fields=3 \
# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR2}/part-00000 | sed -n '7p;8q'
hdfs dfs -rm -r -skipTrash ${OUT_DIR}* > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR2}* > /dev/null

is	126420


rm: `assignment1_1580985846257808*': No such file or directory
20/02/06 10:44:09 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/02/06 10:44:09 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/02/06 10:44:10 INFO mapred.FileInputFormat: Total input files to process : 1
20/02/06 10:44:10 INFO mapreduce.JobSubmitter: number of splits:2
20/02/06 10:44:10 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
20/02/06 10:44:10 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1580979577582_0005
20/02/06 10:44:11 INFO impl.YarnClientImpl: Submitted application application_1580979577582_0005
20/02/06 10:44:11 INFO mapreduce.Job: The url to track the job: http://6169cd1e4e0b:8088/proxy/application_1580979577582_0005/
20/02/06 10:44:11 INFO mapreduce.Job: Running job: job_1580979577582_0005
20/02/06 10:44:17 INFO mapreduce.Job: Job job_1580979577582_0005 running in uber mode : false
20/02/06 10:44:17